In [153]:
import numpy as np
import pandas as pd
import requests

#sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel

#Librería Spotify
import spotipy
import spotipy.util as util 

#Random
import random

#plotly
import plotly.graph_objects as go

#fuzzywuzzy
from fuzzywuzzy import process

import matplotlib.pyplot as plt  
plt.style.use('dark_background')


In [ ]:
client_id = 'CLIENT_ID'
client_secret = 'CLIENT_SECRET'
user_id='USER_ID'
AUTH_URL = 'https://accounts.spotify.com/api/token'
redirect_uri="http://localhost:8080"
username='USERNAME'
access_token = connect_spotify(client_id, client_secret, AUTH_URL)

In [82]:
def connect_spotify(client_id, client_secret, AUTH_URL):
    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    # convert the response to JSON
    auth_response_data = auth_response.json()
    # save the access token
    access_token = auth_response_data['access_token']
    return access_token

In [84]:
#Acces to modify the playlist
scope="playlist-modify-public playlist-modify-private ugc-image-upload"

token=util.prompt_for_user_token(username,scope,client_id,client_secret,redirect_uri)
sp = spotipy.Spotify(auth=token)

In [85]:
#Function to obtain the features of a song.
def song_features(track):
    headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}
    BASE_URL = 'https://api.spotify.com/v1/'
    track_id = track
    r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
    response_features=r.json()
    return pd.DataFrame([response_features])

In [86]:
#Function to get the id's songs from playlist. 
def get_id_songs(playlist):    
    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)
    }
    BASE_URL = 'https://api.spotify.com/v1/'
    r = requests.get(BASE_URL + 'playlists/'+playlist+'/', headers=headers)
    response_tracks=r.json()
    lst=[]
    lst = [i['track']['id'] for i in response_tracks['tracks']['items']]
    return(lst)

In [87]:
#Function to get the features of the songs from a playlist.
def playlist_features(playlist):
    songs = get_id_songs(playlist)
    df_features = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
        'time_signature'])
    for i in songs:
        track=i
        features = song_features(track)
        total=[df_features, features]
        df_features=pd.concat(total)
    return df_features

In [88]:
#Function to get the features of the songs from database.
def all_playlist_features():
    songs = get_id_songs(playlist)
    global track 
    while track in songs: 
        songs.remove(track)
    df_features = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
        'time_signature'])
    for i in songs:
        track=i
        features = song_features(track)
        total=[df_features, features]
        df_features=pd.concat(total)
    return df_features

I select a random element from a list.

In [89]:
def selectRandom(names):
  return random.choice(names)

Choose your mood

In [131]:
enter = input("What rhythm do you want today? Please choose from: 'Happy', 'Booster', 'Sad', 'Relax', 'Motivation', 'Energy','Dancing', 'Drive', 'Mix' or create your own playlist by setting 'Own' :  ")
ok= ['Happy', 'Booster', 'Sad', 'Relax', 'Motivation', 'Energy','Dancing', 'Drive', 'Mix','Own']
feeling=process.extractOne(enter, ok, score_cutoff=80)[0]

if feeling == 'Own':
    artist=input("artist:  ")
    track=input("track:  ") 
    track_id = sp.search(q='artist:' + artist + ' track:' + track, type='track')
    ID = pd.DataFrame([track_id['tracks']['items'][0]['id']])
    ID=ID.iloc[0][0]
elif feeling == 'Happy':
    songs = get_id_songs('37i9dQZF1DXdPec7aLTmlC')
    ID = selectRandom(songs)
elif feeling == 'Booster':
    songs = get_id_songs('37i9dQZF1DX3rxVfibe1L0')
    ID = selectRandom(songs)
elif feeling == 'Sad':
    songs = get_id_songs('37i9dQZF1DWZqdNJSufucb')
    ID = selectRandom(songs)
elif feeling == 'Relax':
    songs = get_id_songs('37i9dQZF1DWVIzZt2GAU4X')
    ID = selectRandom(songs)
elif feeling == 'Motivation':
    songs = get_id_songs('37i9dQZF1DXdxcBWuJkbcy')
    ID = selectRandom(songs)
elif feeling == 'Energy':
    songs = get_id_songs('37i9dQZF1DWYp5sAHdz27Y')
    ID = selectRandom(songs)
elif feeling == 'Mix':
    songs = get_id_songs('37i9dQZEVXcOx8nqzmzXjg')
    ID = selectRandom(songs)
elif feeling == 'Dancing':
    songs = get_id_songs('37i9dQZF1DWSX4baQVDQut')
    ID = selectRandom(songs)
elif feeling == 'Drive':
    songs = get_id_songs('37i9dQZF1DX9wC1KY45plY')
    ID = selectRandom(songs)
else:
    feeling = input("Please choose between: 'Happy', 'Booster', 'Sad', 'Relax', 'Motivation', 'Energy','Dancing', 'Drive', 'Mix' or create your own playlist by setting 'Own' :  ")

Features of song


In [93]:
df_features= song_features(ID)
df_features.drop(['type','uri','track_href','analysis_url','duration_ms'], axis = 'columns', inplace=True)
df_features.set_index('id', inplace = True)
df_features=df_features.astype('float64')
df_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
id,,,,,,,,,,,,
1WCEAGGRD066z2Q89ObXTq,0.601,0.713,4.0,-3.758,0.0,0.0449,0.0282,0.0,0.158,0.464,122.872,4.0


In [94]:
data_songs=pd.read_csv('datasets/data_songs_clean.csv').set_index('id', drop=True)

In [95]:
#Eliminar las canciones elegidas como muestra del df global.
for song in df_features.index:
    if song in data_songs.index:
        data_songs=data_songs.drop(index=song)
        

In [96]:
#Concateno

frames = [data_songs, df_features]

#concatenate dataframes
df_mix = pd.concat(frames, sort=False)

In [97]:
#Elimino los nulos.
df_mix.dropna(inplace=True)

In [98]:
#Extraigo solo las features
df_mix_values= df_mix.iloc[:,1:].values

In [99]:
#Extandarizo los df. Media 0 y desviación típica 1
scaler = StandardScaler()
df_mix_scaled = scaler.fit_transform(df_mix_values)

In [100]:
#Normalización:

#Calculo de la magnitud del Vector para normalizar el vector.
data_songs_mag = np.sqrt((df_mix_scaled*df_mix_scaled).sum(axis=1))

ndata_song=df_mix_scaled.shape[0]
normalized_data_songs=df_mix_scaled/data_songs_mag.reshape(ndata_song,1)

In [101]:
cos_sim = linear_kernel(normalized_data_songs,normalized_data_songs[-1:])
cos_sim.shape

(6958, 1)

In [102]:
print(cos_sim[2,0])

-0.4583882135960509


In [103]:
cos_sim_df = pd.DataFrame(cos_sim)

In [104]:
songs_ponderate = pd.concat([df_mix.reset_index(), cos_sim_df], axis=1)

In [105]:
songs_ponderate.rename(columns={0: 'similarity'}, inplace=True)
similar_song = songs_ponderate.sort_values(by=['similarity'],ascending=False)
similar_song = similar_song.drop(similar_song[similar_song['similarity']=='1'].index)

In [106]:
best_tracks=similar_song['id'].head(50)

In [107]:
tracks = best_tracks.values.tolist()

In [108]:
tracks=["spotify:track:" + a for a in tracks]

Create New Playlist

In [109]:
playlist_name=feeling+' Playlist'
playlist_info= sp.user_playlist_create(user_id, name=playlist_name, public=False, description='This is a playlist based on metrics such as: danceability, energy, loudness, acousticness, instrumentalness.  Enjoy it!')
playlist_id=playlist_info['id']

Change the cover image

In [110]:
sp.playlist_upload_cover_image(playlist_id, image_b64="...")

In [111]:
sp.playlist_add_items(playlist_id=playlist_id, items=tracks, position=None)

{'snapshot_id': 'NCwxODU5ZWE2ZTEwMzU1OTI0NWJiZTg4NjA1YjI3NTE2MTdmYzBkZDY0'}

In [112]:
happy_playlist= '37i9dQZF1DXdPec7aLTmlC'
happy_df = playlist_features(happy_playlist)
happy_df.drop(['type','uri','track_href','analysis_url','duration_ms'], axis = 'columns', inplace=True)
happy_df.set_index('id', inplace = True)
happy_df=happy_df.astype('float64')
happy_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
id,,,,,,,,,,,,
6bQfNiqyCX7UaQSvVVGo4I,0.788,0.859,2.0,-2.724,1.0,0.0856,0.28100,0.000000,0.0424,0.822,141.020,4.0
2Xy1UIVfOlukyye6Fd0wrI,0.673,0.729,6.0,-4.879,1.0,0.0496,0.08300,0.000000,0.1000,0.675,122.023,4.0
5ri4zqtWhG07hIuNNDWP76,0.806,0.513,4.0,-6.208,1.0,0.0685,0.21700,0.000000,0.2170,0.503,124.988,4.0
5K6Ssv4Z3zRvxt0P6EKUAP,0.595,0.784,1.0,-4.878,1.0,0.1020,0.21600,0.000000,0.0608,0.479,124.111,4.0
3LEgGAl9zLcQWGTp7ptqtQ,0.621,0.747,11.0,-5.084,1.0,0.0337,0.22200,0.000194,0.1470,0.426,124.015,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1z1XegUChLLQfcbtIK8ABn,0.686,0.830,8.0,-4.437,1.0,0.0369,0.09480,0.000001,0.1820,0.486,123.032,4.0
2xytFpmHqcz6UHp1sh0SoR,0.722,0.590,2.0,-7.512,1.0,0.0435,0.02640,0.000000,0.0793,0.368,104.931,4.0
7CTdh15dTO5HqoeqRPHFha,0.696,0.792,0.0,-5.659,1.0,0.0494,0.11300,0.000000,0.0564,0.445,121.962,4.0


In [113]:
relax_playlist= '37i9dQZF1DWVIzZt2GAU4X'
relax_df = playlist_features(relax_playlist)
relax_df.drop(['type','uri','track_href','analysis_url','duration_ms'], axis = 'columns', inplace=True)
relax_df.set_index('id', inplace = True)
relax_df=relax_df.astype('float64')
relax_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
id,,,,,,,,,,,,
0gplL1WMoJ6iYaPgMCL0gX,0.604,0.366,5.0,-7.519,1.0,0.0282,0.578,0.000000,0.1330,0.130,141.981,4.0
0zn5Sa8fRjpUo7XXUaNRTB,0.604,0.387,9.0,-6.879,0.0,0.0314,0.841,0.000000,0.1090,0.455,113.837,4.0
5FKnuwsKf7YFpmhA5Wnl0q,0.443,0.540,4.0,-8.536,0.0,0.0400,0.697,0.000010,0.1020,0.133,119.161,4.0
7dZ1Odmx9jWIweQSatnRqo,0.666,0.423,0.0,-8.012,1.0,0.0430,0.494,0.000000,0.1060,0.154,129.890,4.0
7LyBwug9tJgbaEzRCqtY78,0.589,0.471,10.0,-4.908,1.0,0.0262,0.674,0.000000,0.1050,0.262,95.972,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1r7J0ELqOw62KVlgjrmKnw,0.455,0.204,0.0,-10.737,1.0,0.0336,0.782,0.000000,0.1030,0.370,80.364,4.0
3WDxNaUMRsWk1aoDkz8N9x,0.741,0.275,8.0,-12.322,1.0,0.0632,0.848,0.000000,0.0881,0.424,129.939,4.0
4WzhjxvLP95y7AMDy0Atwb,0.481,0.267,7.0,-9.249,1.0,0.0339,0.800,0.000000,0.2490,0.137,117.640,3.0


In [114]:
happy_df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0
mean,0.673860,0.744190,5.750000,-5.117380,0.770000,0.067686,0.110317,0.000840,0.177680,0.557340,118.004350,4.0
std,0.095434,0.111068,3.525878,1.383293,0.422953,0.051047,0.095401,0.008059,0.128001,0.190817,18.312604,0.0
min,0.333000,0.419000,0.000000,-8.173000,0.000000,0.029500,0.000618,0.000000,0.024600,0.132000,82.959000,4.0
25%,0.616250,0.673000,2.000000,-6.073500,1.000000,0.039700,0.030675,0.000000,0.089500,0.430500,105.708750,4.0
50%,0.686000,0.755000,6.000000,-5.051000,1.000000,0.048850,0.072900,0.000000,0.145500,0.547500,118.500500,4.0
75%,0.739500,0.830000,9.000000,-4.149750,1.000000,0.069925,0.181250,0.000003,0.218250,0.702750,124.013500,4.0
max,0.929000,0.934000,11.000000,-1.856000,1.000000,0.360000,0.412000,0.080600,0.687000,0.956000,180.007000,4.0


In [115]:
relax_df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,75.000000,75.000000,75.000000,75.000000,75.00000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,0.514920,0.326356,4.786667,-8.492427,0.88000,0.041647,0.753384,0.000377,0.134463,0.283373,117.976240,3.800000
std,0.109985,0.123882,3.853020,2.163629,0.32715,0.021997,0.218015,0.002978,0.106450,0.117205,28.102967,0.592749
min,0.270000,0.030700,0.000000,-17.636000,0.00000,0.025300,0.003820,0.000000,0.056300,0.086300,71.021000,1.000000
25%,0.430000,0.259500,1.000000,-9.492000,1.00000,0.033650,0.685500,0.000000,0.099800,0.191500,94.137000,4.000000
50%,0.501000,0.305000,5.000000,-8.335000,1.00000,0.036500,0.841000,0.000000,0.110000,0.274000,120.802000,4.000000
75%,0.605500,0.400500,8.500000,-7.030500,1.00000,0.040650,0.907500,0.000003,0.130500,0.372500,136.919500,4.000000
max,0.741000,0.720000,11.000000,-3.879000,1.00000,0.198000,0.967000,0.025800,0.764000,0.551000,172.152000,5.000000


In [154]:
for column in relax_df.columns:
    y0 = happy_df[column]
    y1 = relax_df[column]

    fig = go.Figure()
    fig.add_trace(go.Box(y=y0, name="Happy_"+column,marker_color='Gold'))
    fig.add_trace(go.Box(y=y1, name="Relax_"+column,marker_color='#D194FB'))
    fig.show(title=column)